In [0]:
#Cargamos las librerías
import numpy as np
import bokeh.plotting as bkp
import bokeh.models as bkm
import bokeh.io as bkio
from bokeh.colors import RGB
import pandas as pd
import datetime as dt

bkp.output_notebook()

Loading BokehJS ...

In [0]:
#Obtenemos los datos de anomalías de temperaturas de la página de NOAA
#https://www.ncdc.noaa.gov/monitoring-references/faq/anomalies.php
#Usamos la 5 línea como encabezados (header=4) de las columnas, y leemos el archivo a partir de ahí
temp_anomalia=pd.read_csv("https://www.ncdc.noaa.gov/cag/time-series/global/globe/land_ocean/p12/12/1880-2017.csv", sep=",", header=4,comment='#')

In [0]:
#Leemos las primeras 5 filas para ver que se han cargado bien los datos
temp_anomalia.head(n=5)

,Year,Value
0,188001,-0.00
1,188002,-0.12
2,188003,-0.13
3,188004,-0.05
4,188005,-0.07


In [0]:
#renombramos los encabezados
temp_anomalia.set_axis(['fecha','anomalia'], axis=1, inplace=True)

In [0]:
#transformamos la fecha al formato estándar
temp_anomalia['fecha']=pd.to_datetime(temp_anomalia['fecha'],format="%Y%m")

In [0]:
#Como los datos son mensuales, calculamos el promedio anual (La forma de hacerlo es una convolución con 
#un vector con 12 unos. Posteriormente habría que dividir entre 12, excepto en los bordes, que la convolución toma
#menos datos. Para saber entre que número hay que dividir, uso una columna auxiliar llena de unos a la cual también
#realizo la misma convolución). Suena raro, pero es escueto de programar.

window_size = 12
window = np.ones(window_size)
columna_aux = np.ones(len(temp_anomalia['fecha']))
temp_anomalia['media_anual']=np.convolve(temp_anomalia['anomalia'], window, 'same')/np.convolve(columna_aux, window, 'same')

In [0]:
#Ahora calculamos el promedio a 10 años con el mismo procedimiento.
window_size = 120
window = np.ones(window_size)
columna_aux = np.ones(len(temp_anomalia['fecha']))
temp_anomalia['media_10años']=np.convolve(temp_anomalia['anomalia'], window, 'same')/np.convolve(columna_aux, window, 'same')

In [0]:
#Cargamos los colores elegidos como paleta para este MOOC
p_azulmarino = RGB(0, 28, 100)
p_azulclaro = RGB(0, 131, 255)
p_amarillo = RGB(244, 255, 84)
p_verde = RGB(172, 255, 165)
p_rojo = RGB(255, 67, 67)

In [0]:
#Creamos una figura de Bokeh que va a contener nuestra gráfica. Indico el tamaño, que el eje X van a ser fechas
#y las herramientas activas (mover la gráfica y hacer zoom con el ratón)

p = bkp.figure(tools=["pan,wheel_zoom,box_zoom,save,reset"],
               width=900, height=500, x_axis_type="datetime", active_drag = "pan", active_scroll = "wheel_zoom")

#Represento los datos mensuales como puntos rojos
#p.circle(temp_anomalia['fecha'], temp_anomalia['anomalia'], size=4, color='red', alpha=0.4, legend='Datos Mensuales')

#Represento el promedio anual como una línea azul
p4=p.line(source=temp_anomalia, x='fecha', y='media_anual', color=p_azulmarino, alpha=0.7, 
       legend='Promedio anual',line_width=3)

p.add_tools(bkm.HoverTool(renderers=[p4], tooltips=[("Anomalía anual","@{media_anual}{0.00} (@fecha{%Y})")],mode='vline', formatters={'fecha':'datetime'}))

#Represento el promedio a 10 años como una línea negra
p.line(temp_anomalia['fecha'], temp_anomalia['media_10años'], color='black', alpha=0.8, 
       legend='Promedio 10 años',line_width=3)

GlyphRenderer(id='2ca1ab25-5e1a-4716-872a-e34254faa449', ...)

In [0]:
#Indico las propiedades del gráfico
p.title.text = "Anomalía de Temperatura respecto a la media 1901-2000"
p.title.text_color = "black"
p.title.text_font = "ADAM CG PRO"
p.title.text_font_style = "normal"
p.title.text_font_size = "16pt"
p.title.align = "center"

p.background_fill_color = "white"
p.background_fill_alpha = 1.

p.xaxis.axis_label = 'Año'
p.xaxis.axis_label_text_font = "ADAM CG PRO"
p.xaxis.axis_label_text_font_style  = "normal"
p.xaxis.axis_label_text_font_size = "12pt"
p.yaxis.axis_label = 'Anomalía (ºC)'
p.yaxis.axis_label_text_font = "ADAM CG PRO"
p.yaxis.axis_label_text_font_style  = "normal"
p.yaxis.axis_label_text_font_size = "12pt"

p.ygrid.grid_line_color=p_azulclaro
p.ygrid.grid_line_alpha=0.25
p.xgrid.grid_line_color=p_azulclaro
p.xgrid.grid_line_alpha=0.0

p.xaxis.major_tick_line_color = p_rojo
p.xaxis.major_tick_line_width = 3
p.xaxis.minor_tick_line_color = p_rojo
p.xaxis.minor_tick_line_width = 1
p.yaxis.major_tick_line_color = p_rojo
p.yaxis.major_tick_line_width = 3
p.yaxis.minor_tick_line_color = p_rojo
p.yaxis.minor_tick_line_width = 1

p.outline_line_width = 7
p.outline_line_alpha = 0.5
p.outline_line_color = p_azulclaro

p.legend.location = "top_left"
p.legend.click_policy="hide"
p.legend.label_text_font = "ADAM CG PRO"

In [0]:
#bkp.output_file("AnomaliaTemperatura.html", title='Bokeh Plot', mode='cdn', root_dir=None)

#para guardar un svg usando el menú de la gráfica (en lugar de un PNG), descomentar esta línea
p.output_backend = "svg"

bkp.show(p)